# Data Preparation 

In [22]:
import pandas as pd
import json

## Reading Data

In [23]:
raw_df = pd.read_csv('../data/01_raw/roraima_with_procedures.csv')

In [24]:
raw_df.head()

,quantidade_procedimentos,mes,ano,sigla_uf,valor_ato_profissional,id_municipio_estabelecimento_aih,id_municipio_paciente,id_procedimento_principal
0,1,2,2023,RR,1317.32,1400100,140010,408050063
1,2,2,2023,RR,0.00,1400100,140010,408050063
2,2,2,2023,RR,0.00,1400100,140010,408050063
3,1,2,2023,RR,12.39,1400100,140010,408050063
4,2,2,2023,RR,0.00,1400100,140010,408050063


In [25]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459018 entries, 0 to 2459017
Data columns (total 8 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   quantidade_procedimentos          int64  
 1   mes                               int64  
 2   ano                               int64  
 3   sigla_uf                          object 
 4   valor_ato_profissional            float64
 5   id_municipio_estabelecimento_aih  int64  
 6   id_municipio_paciente             int64  
 7   id_procedimento_principal         int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 150.1+ MB


# Converting Ids to Strings

In [26]:
raw_df['id_municipio_estabelecimento_aih'] = raw_df['id_municipio_estabelecimento_aih'].astype(str)
raw_df['id_municipio_paciente'] = raw_df['id_municipio_paciente'].astype(str)
raw_df['id_procedimento_principal'] = raw_df['id_procedimento_principal'].astype(str)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459018 entries, 0 to 2459017
Data columns (total 8 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   quantidade_procedimentos          int64  
 1   mes                               int64  
 2   ano                               int64  
 3   sigla_uf                          object 
 4   valor_ato_profissional            float64
 5   id_municipio_estabelecimento_aih  object 
 6   id_municipio_paciente             object 
 7   id_procedimento_principal         object 
dtypes: float64(1), int64(3), object(4)
memory usage: 150.1+ MB


## Removing id_municipio_estabelecimento_aih digit


In [27]:
raw_df['id_municipio_estabelecimento_aih'] = raw_df['id_municipio_estabelecimento_aih'].apply(lambda x: x[:-1])
raw_df

,quantidade_procedimentos,mes,ano,sigla_uf,valor_ato_profissional,id_municipio_estabelecimento_aih,id_municipio_paciente,id_procedimento_principal
0,1,2,2023,RR,1317.32,140010,140010,408050063
1,2,2,2023,RR,0.00,140010,140010,408050063
2,2,2,2023,RR,0.00,140010,140010,408050063
3,1,2,2023,RR,12.39,140010,140010,408050063
4,2,2,2023,RR,0.00,140010,140010,408050063
...,...,...,...,...,...,...,...,...
2459013,12,8,2020,RR,0.00,140010,140010,303140151
2459014,12,8,2020,RR,115.20,140010,140010,303110040
2459015,12,8,2020,RR,96.00,140010,140047,303010223
2459016,12,8,2020,RR,115.20,140010,140010,303100044


## Filtering Cities by States of Roraima and Amazonas

In [28]:
filtered_df =raw_df[
        (raw_df['id_municipio_paciente'].str.startswith('14','13'))]

filtered_df



,quantidade_procedimentos,mes,ano,sigla_uf,valor_ato_profissional,id_municipio_estabelecimento_aih,id_municipio_paciente,id_procedimento_principal
0,1,2,2023,RR,1317.32,140010,140010,408050063
1,2,2,2023,RR,0.00,140010,140010,408050063
2,2,2,2023,RR,0.00,140010,140010,408050063
3,1,2,2023,RR,12.39,140010,140010,408050063
4,2,2,2023,RR,0.00,140010,140010,408050063
...,...,...,...,...,...,...,...,...
2459013,12,8,2020,RR,0.00,140010,140010,303140151
2459014,12,8,2020,RR,115.20,140010,140010,303110040
2459015,12,8,2020,RR,96.00,140010,140047,303010223
2459016,12,8,2020,RR,115.20,140010,140010,303100044


In [29]:
# outliers percentage
outliers_percent = 100*(1-(filtered_df.shape[0]/raw_df.shape[0]))
outliers_percent

0.46018369934665015

In [30]:
filtered_df['id_municipio_paciente'].unique()

array(['140010', '140017', '140045', '140060', '140002', '140040',
       '140047', '140020', '140028', '140050', '140030', '140005',
       '140015', '140070', '140023'], dtype=object)

## Mapping City Names

In [31]:
mapping_df = filtered_df.copy()

city_data = pd.read_json('../data/01_raw/cities.json')
city_data['id'] = city_data['id'].astype(str)

city_names = city_data.set_index('id')['city'].to_dict()
mapping_df['municipio_paciente'] = mapping_df['id_municipio_paciente'].map(city_names)
mapping_df['municipio_atendimento'] = mapping_df['id_municipio_estabelecimento_aih'].map(city_names)


In [32]:
mapping_df['municipio_paciente'].unique()

array(['Boa Vista', 'Canta', 'Pacaraima', 'São Luiz', 'Amajari',
       'Normandia', 'Rorainopolis', 'Caracarai', 'Iracema',
       'Sao Joao da Baliza', 'Mucajai', 'Alto Alegre', 'Bomfim',
       'Uiramuta', 'Caroebe'], dtype=object)

In [33]:
mapping_df['municipio_atendimento'].unique()

array(['Boa Vista', 'Bomfim', 'Normandia', 'São Luiz', 'Caracarai',
       'Sao Joao da Baliza', 'Caroebe', 'Pacaraima', 'Alto Alegre',
       'Rorainopolis', 'Mucajai'], dtype=object)

## Mapping Procedure Names

In [34]:
procedure_names = pd.read_csv('../data/01_raw/procedure_names.csv', dtype=str)
procedure_dict = procedure_names.set_index('CÓDIGO')['PROCEDIMENTO'].to_dict()

mapping_df['procedimento_principal'] = mapping_df['id_procedimento_principal'].map(procedure_dict) # mapping the procedimento
mapping_df['procedimento_principal']



0                      ARTROPLASTIA TOTAL PRIMARIA DO JOELHO
1                      ARTROPLASTIA TOTAL PRIMARIA DO JOELHO
2                      ARTROPLASTIA TOTAL PRIMARIA DO JOELHO
3                      ARTROPLASTIA TOTAL PRIMARIA DO JOELHO
4                      ARTROPLASTIA TOTAL PRIMARIA DO JOELHO
                                 ...                        
2459013        TRATAMENTO DE PNEUMONIAS OU INFLUENZA (GRIPE)
2459014    TRATAMENTO DE MALFORMACOES CONGENITAS DO APARE...
2459015                                                  NaN
2459016    TRATAMENTO DE INTERCORRENCIAS CLINICAS NA GRAV...
2459017    TRATAMENTO DE INTERCORRENCIAS CLINICAS NA GRAV...
Name: procedimento_principal, Length: 2447702, dtype: object

## Dropping Values That Couldn't be Identified

In [35]:
processed_df = mapping_df.copy()
processed_df.isnull().sum()



quantidade_procedimentos                 0
mes                                      0
ano                                      0
sigla_uf                                 0
valor_ato_profissional                   0
id_municipio_estabelecimento_aih         0
id_municipio_paciente                    0
id_procedimento_principal                0
municipio_paciente                       0
municipio_atendimento                    0
procedimento_principal              288785
dtype: int64

In [36]:
# percentage of data to be dropped
100*(processed_df.isnull().sum()['procedimento_principal']/processed_df.shape[0])

11.798209095715082

In [37]:
processed_df.dropna(axis=0, how='any', inplace=True)

In [38]:
processed_df = processed_df.reset_index(drop = True)

In [39]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158917 entries, 0 to 2158916
Data columns (total 11 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   quantidade_procedimentos          int64  
 1   mes                               int64  
 2   ano                               int64  
 3   sigla_uf                          object 
 4   valor_ato_profissional            float64
 5   id_municipio_estabelecimento_aih  object 
 6   id_municipio_paciente             object 
 7   id_procedimento_principal         object 
 8   municipio_paciente                object 
 9   municipio_atendimento             object 
 10  procedimento_principal            object 
dtypes: float64(1), int64(3), object(7)
memory usage: 181.2+ MB


In [40]:
null_counts = processed_df.isnull().sum()
null_counts

quantidade_procedimentos            0
mes                                 0
ano                                 0
sigla_uf                            0
valor_ato_profissional              0
id_municipio_estabelecimento_aih    0
id_municipio_paciente               0
id_procedimento_principal           0
municipio_paciente                  0
municipio_atendimento               0
procedimento_principal              0
dtype: int64

## Dropping converted columns


In [41]:
processed_df.drop(axis=1, 
                  columns=['id_municipio_estabelecimento_aih', 
                           'id_municipio_paciente',
                           'id_procedimento_principal'],
                  inplace=True)

processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158917 entries, 0 to 2158916
Data columns (total 8 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   quantidade_procedimentos  int64  
 1   mes                       int64  
 2   ano                       int64  
 3   sigla_uf                  object 
 4   valor_ato_profissional    float64
 5   municipio_paciente        object 
 6   municipio_atendimento     object 
 7   procedimento_principal    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 131.8+ MB


## Saving Processed Data

In [42]:
processed_df.to_csv('../data/02_processed/processed_data.csv', index=False)